In [2]:
import numpy as np
import pandas as pd
import json
import os
import openpyxl

In [3]:
sample_data = {
    "match_id": [],
    "season": [],
    "balls_per_over": [],
    "city": [],
    "dates": [],
    "match_referees": [],
    "reserve_umpires": [],
    "tv_umpires": [],
    "umpire1": [],
    "umpire2": [],
    "winner": [],
    "result": [],
    "player_of_match": [],
    "toss_decision": [],
    "toss_winner": [],
    "dl_applied": [],
    "venue": [],
    "team_a": [],
    "team_b": [],
    "target_overs": [],
    "target_runs": [],
    "win_by_runs": [],
    "win_by_wickets": [],
}

# Set the folder path
folder_path = 'ipl_teamAvsB'

# Initialize match counter
match_counter = 1

# Loop through files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        # Construct the file path
        file_path = os.path.join(folder_path, filename)
        
        # Open the JSON file
        with open(file_path, 'r') as file:
            # Load JSON data
            data = json.load(file)
            
            # Append match_id
            sample_data["match_id"].append(match_counter)
            
            # Append other data
            sample_data["balls_per_over"].append(data["info"]["balls_per_over"])
            sample_data["city"].append(data["info"].get("city", ""))
            sample_data["dates"].append(data["info"]["dates"][0])
            sample_data["match_referees"].append(data["info"]["officials"]["match_referees"][0] if data["info"]["officials"]["match_referees"] else "")
            sample_data["reserve_umpires"].append(data["info"]["officials"].get("reserve_umpires", [""])[0])
            sample_data["tv_umpires"].append(data["info"]["officials"].get("tv_umpires", [""])[0])
            sample_data['winner'].append(data["info"]["outcome"].get('winner', ""))
            sample_data["player_of_match"].append(", ".join(data["info"].get("player_of_match", [])))
            sample_data["toss_decision"].append(data["info"]["toss"]["decision"])
            sample_data["toss_winner"].append(data["info"]["toss"]["winner"])
            sample_data["venue"].append(data["info"]["venue"])
            sample_data["team_a"].append(data["info"]["teams"][0])
            sample_data["team_b"].append(data["info"]["teams"][1])
            
            umpires = data["info"]["officials"]["umpires"]
            if len(umpires) >= 2:
                sample_data["umpire1"].append(umpires[0])
                sample_data["umpire2"].append(umpires[1])
            else:
                sample_data["umpire1"].append("")
                sample_data["umpire2"].append("")

            innings_data = data.get("innings", [])
            if len(innings_data) > 1:
                target_overs = innings_data[1].get("target", {}).get("overs", 0)
                target_runs = innings_data[1].get("target", {}).get("runs", 0)
                sample_data["target_overs"].append(target_overs)
                sample_data["target_runs"].append(target_runs)
            else:
                sample_data["target_overs"].append(0)
                sample_data["target_runs"].append(0)
            
            win_by_runs = data["info"]["outcome"].get("by", {}).get("runs", 0)
            win_by_wickets = data["info"]["outcome"].get("by", {}).get("wickets", 0)
            sample_data["win_by_runs"].append(win_by_runs)
            sample_data["win_by_wickets"].append(win_by_wickets)

            method = data["info"]["outcome"].get("method", "")
            if method == "D/L":
                sample_data["dl_applied"].append(1)
            else:
                sample_data["dl_applied"].append(0)

            result = data["info"]["outcome"].get("result", "normal")
            sample_data["result"].append(result)

            # Fetch the date and append to the season column
            date = data["info"]["dates"][0]
            year = pd.to_datetime(date).year
            sample_data["season"].append(f"IPL-{year}")
            
            # Increment the match counter
            match_counter += 1

# Convert sample_data dictionary to DataFrame
df_matches = pd.DataFrame(sample_data)
df_matches


,match_id,season,balls_per_over,city,dates,match_referees,reserve_umpires,tv_umpires,umpire1,umpire2,...,toss_decision,toss_winner,dl_applied,venue,team_a,team_b,target_overs,target_runs,win_by_runs,win_by_wickets
0,1,IPL-2017,6,Hyderabad,2017-04-05,J Srinath,N Pandit,A Deshmukh,AY Dandekar,NJ Llong,...,field,Royal Challengers Bangalore,0,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,20.0,208,35,0
1,2,IPL-2017,6,Pune,2017-04-06,M Nayyar,Navdeep Singh,VK Sharma,A Nand Kishore,S Ravi,...,field,Rising Pune Supergiant,0,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,20.0,185,0,7
2,3,IPL-2017,6,Rajkot,2017-04-07,V Narayan Kutty,K Srinivasan,YC Barde,Nitin Menon,CK Nandan,...,field,Kolkata Knight Riders,0,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,20.0,184,0,10
3,4,IPL-2017,6,Indore,2017-04-08,Chinmay Sharma,R Pandit,KN Ananthapadmanabhan,AK Chaudhary,C Shamshuddin,...,field,Kings XI Punjab,0,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,20.0,164,0,6
4,5,IPL-2017,6,Bengaluru,2017-04-08,J Srinath,Navdeep Singh,A Nand Kishore,S Ravi,VK Sharma,...,bat,Royal Challengers Bangalore,0,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,20.0,158,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1020,IPL-2016,6,Raipur,2016-05-22,Chinmay Sharma,YC Barde,VK Sharma,A Nand Kishore,BNJ Oxenford,...,field,Royal Challengers Bangalore,0,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Royal Challengers Bangalore,20.0,139,0,6
1020,1021,IPL-2016,6,Bangalore,2016-05-24,RS Mahanama,A Deshmukh,BNJ Oxenford,AK Chaudhary,HDPK Dharmasena,...,field,Royal Challengers Bangalore,0,M Chinnaswamy Stadium,Gujarat Lions,Royal Challengers Bangalore,20.0,159,0,4
1021,1022,IPL-2016,6,Delhi,2016-05-25,RS Mahanama,Navdeep Singh,CK Nandan,M Erasmus,C Shamshuddin,...,field,Kolkata Knight Riders,0,Feroz Shah Kotla,Sunrisers Hyderabad,Kolkata Knight Riders,20.0,163,22,0
1022,1023,IPL-2016,6,Delhi,2016-05-27,RS Mahanama,Navdeep Singh,C Shamshuddin,M Erasmus,CK Nandan,...,field,Sunrisers Hyderabad,0,Feroz Shah Kotla,Gujarat Lions,Sunrisers Hyderabad,20.0,163,0,4


In [9]:
dfnew = df_matches[df_matches['match_id']==34]
dfnew[ ['winner','result',
    'player_of_match',
    'toss_decision',
    'toss_winner',
    'dl_applied',
    'team_a',
    'team_b',
    'target_overs',
    'target_runs',
    'win_by_runs']]

,winner,result,player_of_match,toss_decision,toss_winner,dl_applied,team_a,team_b,target_overs,target_runs,win_by_runs
33,,tie,KH Pandya,bat,Gujarat Lions,0,Gujarat Lions,Mumbai Indians,20.0,154,0
